In [ ]:
!git remote add origin https://github.com/cyberedjs/house-price-web-server.git

fatal: remote origin already exists.


In [ ]:
!git add flask-app.ipynb
!git add mlp_v0.1.h5
!git add submit_form.html
!git add x.csv
!git add y.npy

In [ ]:
!git commit -m "first commit"

On branch main
Untracked files:
	.config/
	drive/
	sample_data/

nothing added to commit but untracked files present


In [ ]:
import numpy as np
import pandas as pd

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

from sklearn.preprocessing import MinMaxScaler

from flask import Flask, render_template, request
from flask_ngrok import run_with_ngrok

x = pd.read_csv('x.csv')

with open('y.npy', 'rb') as f:
  y = np.load(f)

x = x[['OverallQual', 'GrLivArea', 'GarageCars', 'GarageArea', 'TotalBsmtSF', '1stFlrSF', 'FullBath', 'LotShape_rank']]
x_min_max_scaler = MinMaxScaler()
x_min_max_scaler.fit(x)
scaled_x = x_min_max_scaler.transform(x)

y_min_max_scaler = MinMaxScaler()
y_min_max_scaler.fit(y)
scaled_y = y_min_max_scaler.transform(y)

reconstructed_model = keras.models.load_model("mlp_v0.1.h5")

app = Flask(__name__, template_folder='/content')
run_with_ngrok(app)

def preprocess_data(data):
  # return np.zeros((1, 8)) # dummy data

  """
  Dictionary --> np.array (1,8)

  OverallQual 2
  GrLivArea 5000
  GarageCars 2
  GarageArea 480
  TotalBsmtSF 991
  1stFlrSF 1087
  FullBath 2
  LotShape IR3 --> 1, 2, 3, 4
  """

  #Scale Normalization

  x = []
  for k, v in data.items():
    if k == "LotShape":
      if v == "Reg":
        x.append(4)
      elif v == "IR3":
        x.append(3)
      elif v == "IR2":
        x.append(2)
      elif v == "IR1":
        x.append(1)
    else:
      x.append(float(v))

  x = np.array(x)
  x = x.reshape((1, -1))

  #min max scaling
  scaled_x = x_min_max_scaler.transform(x)
  print(scaled_x.shape)

  return scaled_x


@app.route("/")
def predict():

  return render_template("submit_form.html")

@app.route('/result', methods=['POST'])
def result():

  data = request.form

  message = ''
  message += "<hl>House Price</h1>"

  for k, v in data.items():
    print(k, v)
    message += k + ": " + v + "</br>"

  x = preprocess_data(data)
  pred = reconstructed_model.predict(x)
  pred = y_min_max_scaler.inverse_transform(pred)

  message += "</br>"
  message += "Predicted price: " + str(pred[0][0])

  return message

app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://775c5db2e605.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


127.0.0.1 - - [14/Aug/2021 13:48:11] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [14/Aug/2021 13:48:11] "GET /favicon.ico HTTP/1.1" 404 -


OverallQual 6
GrLivArea 1464
GarageCars 2
GarageArea 480
TotalBsmtSF 991
1stFlrSF 1087
FullBath 2
LotShape IR1
(1, 8)


127.0.0.1 - - [14/Aug/2021 13:48:13] "POST /result HTTP/1.1" 200 -
